Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels, Random
Random.seed!(12345)

MersenneTwister(UInt32[0x00003039], Random.DSFMT.DSFMT_state(Int32[-870096391, 1072918504, -1812426662, 1073255081, -733866021, 1073404543, 807620846, 1073368448, 1919433844, 1072852359  …  -362113007, 1073100625, -166402106, 1073460158, -1907020342, 721295190, -750225566, -1300227565, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[!, :pulled_left] = convert(Array{Int64}, df[!, :pulled_left])
df[!, :prosoc_left] = convert(Array{Int64}, df[!, :prosoc_left])
df[!, :condition] = convert(Array{Int64}, df[!, :condition])
df[!, :actor] = convert(Array{Int64}, df[!, :actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_actors = length(unique(df[!, :actor]))
X = hcat(ones(Int64, N), df[!, :prosoc_left] .* df[!, :condition]);
A = df[!, :actor]
y = df[!, :pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [7]:
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

#import Zygote
#∇P = LogDensityRejectErrors(ADgradient(:Zygote, P));
#∇P = ADgradient(:Zygote, P);

Tune and sample.

In [8]:
@time chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.013 s/step ...done
MCMC, adapting ϵ (25 steps)
0.022 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0048 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0039 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0035 s/step ...done
MCMC, adapting ϵ (400 steps)
step 367 (of 400), 0.0027 s/step
0.0027 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0033 s/step ...done
MCMC (1000 steps)
step 345 (of 1000), 0.0029 s/step
step 707 (of 1000), 0.0028 s/step
0.0029 s/step ...done
  8.298274 seconds (42.19 M allocations: 4.935 GiB, 12.81% gc time)


We use the transformation to obtain the posterior from the chain.

In [9]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [4.805608442141108, 0.2389779757501151], α = [-5.4228120431192135, 21.102838682057858, -5.087605475240926, -5.339686571980073, -5.239935978535041, -4.187801436108223, -2.196616929378975]) 
 (β = [3.1779855333330698, 0.5965490813806572], α = [-3.332679123628399, 22.4757391092144, -4.052102062224823, -4.302760278141705, -3.5144792187419234, -2.9698061668305353, -0.9186563627898878])
 (β = [3.6751239531827453, 0.5510236952460794], α = [-4.006795241697499, 17.439170211065864, -4.428966426100576, -4.43289953739397, -3.92856451258628, -3.195481244391613, -1.7185600988784437])  
 (β = [6.811143293480398, 0.2662578097605355], α = [-7.316318256645592, 2.853053665655831, -7.543343290655876, -7.55305997384782, -7.380809100313019, -6.3777111648238165, -4.189699859996347])   
 (β = [-4.431487462956057, 0.45030287799194935], α = [4.0152969847455955, 12.178060714384907, 3.4452039433839743, 3.6231689167024097, 3.60

Extract the parameter posterior means: `β`,

In [10]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.451237768661092 
 0.4233537383099833

Extract the parameter posterior means: `β`,

In [11]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -1.9000576019113276
 10.145744904561283 
 -2.194854907777116 
 -2.183382830567691 
 -1.9048052110036864
 -0.971437677726311 
  0.5935301675225381

Effective sample sizes (of untransformed draws)

In [12]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 393.489  1000.0  399.712  261.483  …  394.444  403.401  395.341  396.188

NUTS-specific statistics

In [13]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.0 0.89 0.96 0.99 1.0
  termination: AdjacentDivergent => 2% AdjacentTurn => 32% DoubledTurn => 66%
  depth: 0 => 0% 1 => 0% 2 => 2% 3 => 58% 4 => 39% 5 => 1% 6 => 0%


Result rethinking

In [14]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [15]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.451237768661092, 0.4233537383099833]                                                                                                       
 [-1.9000576019113276, 10.145744904561283, -2.194854907777116, -2.183382830567691, -1.9048052110036864, -0.971437677726311, 0.5935301675225381]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*